In [2]:
import os
import glob
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from langchain_community.embeddings import HuggingFaceEmbeddings  
from openai import OpenAI
from langchain_community.chat_models import ChatOllama


In [3]:
import gradio as gr
from dotenv import load_dotenv
import fitz  # PyMuPDF
import re
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain


c:\Users\aoujjet\OneDrive - Grupo Sampol\Escritorio\Proyectos LLM\ChatWithPDF\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain.memory import ConversationBufferMemory


Subir y leer PDF

In [5]:
def cargar_pdf(path_pdf):
    texto_completo = ""
    doc = fitz.open(path_pdf)
    for pagina in doc:
        texto_completo += pagina.get_text()
    doc.close()
    return texto_completo

# Simulamos la subida
pdf_path = r"Produccion_Nuevo_Gemelo.pdf"  # Debes tener este archivo en el mismo directorio que tu notebook
texto_pdf = cargar_pdf(pdf_path)

# Mostramos los primeros 500 caracteres
print(texto_pdf[:500])


En este archivo se recopilan todos los pasos y modificaciones necesarias para 
poner en marcha un GD desde cero. 
**Nota: Leer el archivo completo antes de comenzar a implementar** 
Configuración del Servidor Ubuntu (si es necesario) 
1. Instalación de Programas: 
• 
Se deben instalar los programas necesarios en el servidor Ubuntu 
según los requisitos del proyecto. 
2. Localización de la Tarjeta de Red: 
• 
Se debe identificar y localizar la tarjeta de red del servidor para 
configurar la conex


dividir texto en chunks

In [6]:
# Convertir el texto en documentos
documento = Document(page_content=texto_pdf)


In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=400)
chunks = text_splitter.split_documents([documento])
len(chunks)


8

Embedding y vectores

In [8]:
db_name = "ChatVectors"

In [9]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


C:\Users\aoujjet\AppData\Local\Temp\ipykernel_7780\102896465.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [10]:
# vaciar la BD si existe 
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()


In [11]:
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 8 documents


Modelo LLM

In [12]:
MODEL = "gemma3:4b" 

# 1) llm
llm = ChatOllama(model=MODEL)

# 2) set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# 3) the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever()

# 4) putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

C:\Users\aoujjet\AppData\Local\Temp\ipykernel_7780\641253344.py:4: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model=MODEL)
C:\Users\aoujjet\AppData\Local\Temp\ipykernel_7780\641253344.py:7: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [13]:
retriever.get_relevant_documents("172.30.10.152:9443")

C:\Users\aoujjet\AppData\Local\Temp\ipykernel_7780\2065357034.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("172.30.10.152:9443")


[Document(id='fe38605e-8178-48cf-8e1c-9730670a5f50', metadata={}, page_content='• \nAñadir DB.cfg de la API (donde esta DB_template.cfg)  \n• \nAñadir en scheduler, mongo.cfg y añadir carpeta taglists en \nscheduler de la API \n• \nModificar la IP del backfill en producer/backfill_stream.py \n➢ Arrancar el productor Kafka \n➢ Arrancar la API docker tween-streaming-api -d en \nTweenStremingApi \n➢ Arrancar docker completo de TweenStremingApi (para ver \nerrores) \n➢ Abrir portainer para ver que todo ha ido bien: 172.30.10.152:9443 \npara ver logs y reiniciar contenedores etc. \n➢ Abrir Kafka para ver si llegan mensajes 172.30.10.152:8080/ \n \n4. Modificar en GDDataPlataform, el repositorio TweenDataLake: \n• \nCrear la base de datos en Dbeaver (Timescale).  \n1. Host 172.30.10.249 \n2. port 5432 \n3. usr y pw: root  \n4. Nombre de la DB: el nombre de la central igual que en \nmongo. \n• \nAñadir archivo db/init.sql. Editarlo y añadir las 3 filas que hay para \ncada GD.  \n• \nEn TweenD

Chat

In [14]:
def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

Gradio IG

In [15]:
# view = gr.ChatInterface(chat, type="messages").launch(inbrowser=False)


In [ ]:
# Crear el interfaz de chat
view = gr.ChatInterface(chat, type="messages")

# Lanza el servidor en 0.0.0.0:7860 para que esté accesible externamente
view.launch(
    inbrowser=False,
    share=False,
    server_name="0.0.0.0",
    server_port=7860
)


Running on local URL:  http://0.0.0.0:7860

To create a public link, set `share=True` in `launch()`.
